# Data Cleansing and Transformation
**Team:** The Closer  
**Member:** Rongala Sreedhar  

## Problem Description
The goal is to prepare the Healthcare dataset for analysis by addressing data quality issues. We will demonstrate techniques for:
1. Handling Missing Values (Imputation)
2. Handling Outliers
3. NLP Featurization and Cleaning


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re

# Load Data
df = pd.read_excel('Healthcare_dataset (1).xlsx', sheet_name='Dataset')
print("Original Shape:", df.shape)

## 1. Data Corruption (Simulation)
Since the dataset has 0 missing values, we introduce synthetic NaNs to demonstrate our cleaning techniques.

In [ ]:
np.random.seed(42)
df_corrupted = df.copy()
cols_to_corrupt = ['Dexa_Freq_During_Rx', 'Count_Of_Risks']

for col in cols_to_corrupt:
    mask = np.random.choice([True, False], size=df.shape[0], p=[0.1, 0.9])
    df_corrupted.loc[mask, col] = np.nan
    
print("Missing values after corruption:")
print(df_corrupted[cols_to_corrupt].isna().sum())

## 2. Approach 1: Simple Imputation & Outlier Removal
**Member:** Rongala Sreedhar  
**Technique:** Mean/Mode Imputation + IQR Outlier Removal

In [ ]:
# Create a copy for approach 1
df_app1 = df_corrupted.copy()

# Impute Numeric with Median (Count_Of_Risks)
median_val = df_app1['Count_Of_Risks'].median()
df_app1['Count_Of_Risks'].fillna(median_val, inplace=True)

# Impute Categorical with Mode (Dexa_Freq_During_Rx)
mode_val = df_app1['Dexa_Freq_During_Rx'].mode()[0]
df_app1['Dexa_Freq_During_Rx'].fillna(mode_val, inplace=True)

# Outlier Removal using IQR (on Count_Of_Risks)
Q1 = df_app1['Count_Of_Risks'].quantile(0.25)
Q3 = df_app1['Count_Of_Risks'].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

df_clean_1 = df_app1[(df_app1['Count_Of_Risks'] >= lower) & (df_app1['Count_Of_Risks'] <= upper)]
print(f"Approach 1: Shape after cleaning: {df_clean_1.shape}")

### Peer Review (Simulated)
> **Reviewer:** Example Peer
> *"Good use of Median for count data as it's less sensitive to outliers. IQR is standard but aggressive; check if we lost too many rows."*

## 3. Approach 2: Model-Based Imputation & Transformations
**Member:** (Peer / Alternative Approach)  
**Technique:** KNN Imputation + Log Transformation

In [ ]:
# Create a copy for approach 2
df_app2 = df_corrupted.copy()

# KNN Imputation requires numeric encoding usually, but we'll demonstrate on numeric col
imputer = KNNImputer(n_neighbors=5)
df_app2[['Count_Of_Risks']] = imputer.fit_transform(df_app2[['Count_Of_Risks']])

# Handle Categorical Missing (Dexa_Freq_During_Rx) with 'Missing' category
df_app2['Dexa_Freq_During_Rx'].fillna('Missing_Data', inplace=True)

# Outlier Handling: Log Transformation (instead of removal)
# Adding 1 to avoid log(0)
df_app2['Log_Count_Risks'] = np.log1p(df_app2['Count_Of_Risks'])

print(f"Approach 2: Shape retained: {df_app2.shape}")
print("Log Transform Skewness:", df_app2['Log_Count_Risks'].skew())

## 4. NLP Featurization and Cleaning
**Target Column:** `Ntm_Speciality`

In [ ]:
text_data = df['Ntm_Speciality'].astype(str)

# Regex Cleaning function
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text) # Remove punctuation
    return text

df['Clean_Speciality'] = text_data.apply(clean_text)

# Featurization 1: Count Vectorizer
cv = CountVectorizer(stop_words='english')
cv_matrix = cv.fit_transform(df['Clean_Speciality'])
print("Count Vectorizer Shape:", cv_matrix.shape)

# Featurization 2: TF-IDF
tfidf = TfidfVectorizer(max_features=10)
tfidf_matrix = tfidf.fit_transform(df['Clean_Speciality'])
print("TF-IDF Shape:", tfidf_matrix.shape)
print("Top TF-IDF features:", tfidf.get_feature_names_out())